In [1]:
# === π from a binary substrate: self-contained construction kit ===
# No internet; standard libs only. Charts use matplotlib (no seaborn/colors).
# You can just run this single cell in Colab.

import math, random, itertools, json, os
from decimal import Decimal, getcontext
import numpy as np
import matplotlib.pyplot as plt

# ---------- Config ----------
OUTDIR = "out_pi_from_binary"
os.makedirs(OUTDIR, exist_ok=True)

# Fibonacci word length (bits used for experiments)
N_BITS = 5000
# Random control streams for the polygon test
N_RANDOM_STREAMS = 200
# Triadic polygon depth (n = 3^k)
MAX_K = 8

# High precision for Decimal when we build DF as a decimal-digit number
getcontext().prec = 10000

# ---------- Utilities ----------

def fib_word(n):
    """Return first n bits of the Fibonacci word via morphism 0->01, 1->0 (seed '0')."""
    s = "0"
    while len(s) < n:
        s = s.replace("0", "x").replace("1", "0").replace("x", "01")
    return np.fromiter((1 if c=="1" else 0 for c in s[:n]), dtype=np.uint8)

def thue_morse(n):
    """Thue-Morse bits for comparison."""
    # t(n) = parity of number of 1-bits in n (Walnut construction)
    out = np.zeros(n, dtype=np.uint8)
    for i in range(n):
        out[i] = bin(i).count("1") & 1
    return out

def as_decimal_from_digits(bits):
    """Interpret a 0/1 array as a base-10 decimal 0.b1 b2 b3 ... using Decimal."""
    s = "0." + "".join(str(int(b)) for b in bits)
    return Decimal(s)

def triadic_polygon_bounds(k, R=1.0):
    """
    Return lower/upper bounds on π using regular n-gons (n = 3^k) inscribed/circumscribed.
    For a unit circle: P_in = 2 n sin(pi/n), P_out = 2 n tan(pi/n).
    Then π is between P_in/(2R) and P_out/(2R).
    """
    n = 3**k
    pin = (2.0 * n * math.sin(math.pi / n)) / (2.0*R)
    pout = (2.0 * n * math.tan(math.pi / n)) / (2.0*R)
    return n, pin, pout

def polygon_perimeter_from_angles(angles, R=1.0):
    """Perimeter of polygon formed by points on unit circle at given angles (sorted)."""
    pts = np.column_stack((np.cos(angles), np.sin(angles)))
    # close the polygon
    pts = np.vstack([pts, pts[0]])
    diffs = np.diff(pts, axis=0)
    segs = np.linalg.norm(diffs, axis=1)
    return float(segs.sum())

def binary_chord_pi(bits, M=None, R=1.0):
    """
    Take equally spaced angles on [0, 2π). Keep only steps where bit==1.
    Estimate π from polygon perimeter / (2R).
    If M is None, use number of '1' bits to set spacing so a 'full tour' exists.
    """
    idx_ones = np.flatnonzero(bits)
    m = len(idx_ones) if M is None else int(M)
    if m <= 2:
        return np.nan, m
    # equally spaced angle grid of size  m  (so that ones “sample” m positions)
    # map ones onto the first m grid positions (keeps construction deterministic)
    angles = np.linspace(0.0, 2.0*math.pi, m, endpoint=False)
    perim = polygon_perimeter_from_angles(angles, R=R)
    pi_est = perim / (2.0*R)
    return pi_est, m

def binary_chord_pi_random(n_bits, seed=None, R=1.0):
    rng = random.Random(seed)
    bits = np.fromiter((rng.randint(0,1) for _ in range(n_bits)), dtype=np.uint8)
    return binary_chord_pi(bits, R=R)[0]

def carry_cut_residual(bits, max_k=8):
    """
    Toy 'carry/cut' model:
    - For each triadic level k (n=3^k), take blocks of length 3 from the decimal-digit stream,
      sum them, and interpret (sum/3)/1000 as a micro-cut on the perimeter of the 3^k-gon.
    - Subtract these micro-cuts from the circumscribed bound and see the residual to true π.
    This is exploratory only (no claim of physics content).
    """
    pi_true = math.pi
    block = 3
    res = []
    cursor = 0
    for k in range(1, max_k+1):
        n, pin, pout = triadic_polygon_bounds(k)
        # build cuts until we run out of digits for this level (use a fixed count for comparability)
        n_blocks = min( (len(bits)-cursor) // block, 3 )  # use up to 3 blocks per level
        cut = 0.0
        for _ in range(n_blocks):
            tri = bits[cursor:cursor+block].sum()
            cursor += block
            cut += (tri/3.0)/1000.0   # tiny 'carry-like' decrement (arbitrary scale, fixed rule)
        adj = max(pout - cut, pin)    # never below the inscribed bound
        res.append((k, adj - pi_true))
    return np.array(res)

# ---------- Build sequences ----------
F = fib_word(N_BITS)
TM = thue_morse(N_BITS)
DF = as_decimal_from_digits(F)  # Decimal 0.F

# ---------- 1) Triadic polygon convergence (control) ----------
ks, lo, hi = [], [], []
for k in range(1, MAX_K+1):
    n, pin, pout = triadic_polygon_bounds(k)
    ks.append(k); lo.append(pin); hi.append(pout)

plt.figure(figsize=(7,5))
plt.plot(ks, np.array(hi)-math.pi, marker='o', label='p_out - π (circumscribed)')
plt.plot(ks, np.array(lo)-math.pi, marker='o', label='p_in - π (inscribed)')
plt.axhline(0.0, linestyle='--')
plt.xlabel('k  (n = 3^k)'); plt.ylabel('error relative to π')
plt.title('Triadic regular polygons: bounds to π (control)')
plt.legend(); plt.tight_layout()
plt.savefig(f"{OUTDIR}/triadic_bounds_control.png", dpi=200); plt.close()

# ---------- 2) Binary-chord circle test ----------
# Fibonacci
pi_F, mF = binary_chord_pi(F)
# Thue–Morse (for contrast)
pi_TM, mTM = binary_chord_pi(TM)
# Random controls
random_pis = [binary_chord_pi_random(N_BITS, seed=i) for i in range(N_RANDOM_STREAMS)]

# Plot histogram with markers for F/TM/π
plt.figure(figsize=(7,5))
plt.hist(random_pis, bins=30)
plt.axvline(math.pi, linestyle='--', label='π (true)')
plt.axvline(pi_F, linestyle='-', label=f'Fibonacci estimate ({pi_F:.6f})')
plt.axvline(pi_TM, linestyle='-.', label=f'Thue–Morse estimate ({pi_TM:.6f})')
plt.xlabel('π estimate (perimeter / 2)')
plt.ylabel('count')
plt.title('Binary‑chord circle: Fibonacci vs random streams')
plt.legend(); plt.tight_layout()
plt.savefig(f"{OUTDIR}/binary_chord_hist.png", dpi=200); plt.close()

# ---------- 3) Carry/cut residual vs triadic level ----------
res_F = carry_cut_residual(F, max_k=MAX_K)
res_TM = carry_cut_residual(TM, max_k=MAX_K)

plt.figure(figsize=(7,5))
plt.plot(res_F[:,0], res_F[:,1], marker='o', label='Fibonacci stream')
plt.plot(res_TM[:,0], res_TM[:,1], marker='o', label='Thue–Morse stream')
plt.axhline(0.0, linestyle='--')
plt.xlabel('k  (n = 3^k)')
plt.ylabel('adjusted perimeter − π')
plt.title('“Carry‑cut” residual vs triadic level (toy)')
plt.legend(); plt.tight_layout()
plt.savefig(f"{OUTDIR}/carry_cut_residual.png", dpi=200); plt.close()

# ---------- 4) Print a compact summary ----------
summary = {
    "N_bits": N_BITS,
    "DF_first_60_digits": str(DF)[:62],
    "binary_chord": {
        "pi_true": math.pi,
        "pi_Fibonacci": pi_F,
        "pi_ThueMorse": pi_TM,
        "pi_random_mean": float(np.mean(random_pis)),
        "pi_random_std": float(np.std(random_pis)),
        "m_vertices_F": int(mF),
        "m_vertices_TM": int(mTM)
    },
    "triadic_bounds_last": {
        "k": int(ks[-1]),
        "pin": lo[-1],
        "pout": hi[-1],
        "pin_error": lo[-1]-math.pi,
        "pout_error": hi[-1]-math.pi
    }
}
with open(f"{OUTDIR}/summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print("=== π-from-binary construction kit ===")
print(f"Bits used: N={N_BITS}")
print("First 60 digits of DF (decimal-digit map of Fibonacci word):")
print(str(DF)[:62], "...")
print("\nBinary‑chord π estimates")
print(f"  true π            : {math.pi:.12f}")
print(f"  Fibonacci (F)     : {pi_F:.12f}")
print(f"  Thue–Morse (TM)   : {pi_TM:.12f}")
print(f"  random mean±sd    : {np.mean(random_pis):.12f} ± {np.std(random_pis):.12f}")
print("\nTriadic polygon bounds (control)")
print(f"  k={ks[-1]} (n=3^{ks[-1]}) pin={lo[-1]:.12f}, pout={hi[-1]:.12f}")
print(f"  errors: pin-π={lo[-1]-math.pi:.2e}, pout-π={hi[-1]-math.pi:.2e}")
print(f"\nSaved outputs → {OUTDIR}/")


=== π-from-binary construction kit ===
Bits used: N=5000
First 60 digits of DF (decimal-digit map of Fibonacci word):
0.010010100100101001010010010100100101001010010010100101001001 ...

Binary‑chord π estimates
  true π            : 3.141592653590
  Fibonacci (F)     : 3.141591237041
  Thue–Morse (TM)   : 3.141591826756
  random mean±sd    : 3.141591828783 ± 0.000000024003

Triadic polygon bounds (control)
  k=8 (n=3^8) pin=3.141592533541, pout=3.141592893688
  errors: pin-π=-1.20e-07, pout-π=2.40e-07

Saved outputs → out_pi_from_binary/
